# <span style= "color:crimson"><u>Going Deep!!!</u></span>
<img src = "https://i.pinimg.com/originals/1a/10/6e/1a106e5fa6cb78e4a89bff36fccc3a02.png">


<p style = "font-size:150%"><span style="color:blue">
    We all know, when we talk about <strong>Machine Learning</strong> boosting methods, they kicks the asses of every other ML-Algos when it comes to tabular data
    <br>
    Although, deeplearning has proved to be unreasonalby more effective in fields like Vision, Speech or Language, it haven't shown much effectiveness when it comes to     tabular data. </span>
    
<br>
<span style= "color:crimson"><span style="font-size:150%"><u>Why Go Deeper?</u></span></span>
    
<p style="font-size:150%"><span style="color:blue">Today, we can train machines to detect objects in images, extract meaning from text, stop spam emails, drive cars, discover new drug candidates, and beat top players in Chess, Go, and countless other games and most of the credit goes to deep-learning.
<br>
Machine doesn't understands the science/theory, all it does is generalises the ideas by visiting a lot of data, so lacking a perfect theory we have to rely on intutions. <strong> Deeper Neural Network because it generalises more and overfits less</strong>. <br>
So, even if part of the face is hidden, the network will still pick up a signal from the remaining input, and therefore generalize better. It's a good intuition, and it appears to be what is actually happening. Experiments confirm that deep neural networks outperform shallow ones on common image as well as text tasks.
</span>
</p>

<table><tr><td><img src ="https://pytorch-tabular.readthedocs.io/en/latest/imgs/pytorch_tabular_logo.png"></td>

<td><span style= "color:black"><span style="font-size:350%">The <br>Deep Learning Framework<br>for Tabular Data</span></span></td></tr></table>


 

<p style="font-size:150%">
        As the name suggest <strong>PyTorch Tabular</strong> is a framework based on <a href="https://pytorch.org/">pytorch</a> and <a href="https://www.pytorchlightning.ai/">pytorch lightning</a> which works on Tabular Data.<br>
        Rather than doing everything from scratch, it make out job easy and we can do everything in few simple steps.
    </p>

# <span style="color:crimson"><u>In this tutorial, we'll learn to implement pytorch-tabular in <strong>5-Simple-Steps</strong>:</u></span>
### 
<span style="font-size:150%">
Step 1: Installation<br>
Step 2: Setting up the Configs<br>
Step 3: Initializing the Model and Training<br>
Step 4: Evaluating the Model on unseen data<br>
Step 5: Saving the Model
</span>

In [2]:
import warnings 
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

In [6]:
# reading data
data=pd.read_csv('input/breast-cancer-wisconsion-data/data.csv')
data['diagnosis'] = data['diagnosis'].astype('category').cat.codes# encoding labels
data.drop(['id','Unnamed: 32'],axis=1,inplace=True)

In [7]:
def load_classification_data(df,target_col,test_size):
    torch_data = np.array(df.drop(target_col,axis=1))
    torch_labels =np.array(df[target_col])
    data = np.hstack([torch_data, torch_labels.reshape(-1,1)])
    gen_names = [f"feature_{i}" for i in range(data.shape[-1])]
    col_names = gen_names
    col_names[-1] = "target"
    data = pd.DataFrame(data, columns=col_names)
    cat_col_names = [x for x in gen_names[:-1] if len(data[x].unique())<10]
    num_col_names = [x for x in gen_names[:-1] if x not in [target_col]+cat_col_names]
    test_idx = data.sample(int(test_size * len(data)), random_state=42).index
    test = data[data.index.isin(test_idx)]
    train = data[~data.index.isin(test_idx)]
    return (train, test, ["target"],cat_col_names,num_col_names)

In [8]:
train, test, target_col, cat_col_names, num_col_names= load_classification_data(data,'diagnosis',0.2)

**Sorry in advance for a lot of printing

# Step 1: Installation
***it might depricate some libraries but that will do fine*

In [9]:
# ! pip install pytorch_tabular[all]

# Importing Libraries
and simple data pre-processing formality for pytorch

In [11]:
from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig, NodeConfig, TabNetModelConfig
from pytorch_tabular.config import DataConfig, OptimizerConfig, TrainerConfig, ExperimentConfig
from pytorch_tabular.categorical_encoders import CategoricalEmbeddingTransformer

# Step2: Setting up the Configs:

In [13]:
data_config = DataConfig(
    target=['target'], #target should always be a list. Multi-targets are only supported for regression. Multi-Task Classification is not implemented
    continuous_cols=num_col_names,
    categorical_cols=cat_col_names,
    continuous_feature_transform="quantile_normal",
    normalize_continuous_features=True
)
trainer_config = TrainerConfig(
    auto_lr_find=True, # Runs the LRFinder to automatically derive a learning rate
    batch_size=32,
    max_epochs=100,
    devices=1, #index of the GPU to use. 0, means CPU
)
optimizer_config = OptimizerConfig()
model_config = CategoryEmbeddingModelConfig(
    task="classification",
    layers="4096-4096-512",  # Number of nodes in each layer
    activation="LeakyReLU", # Activation between each layers
    learning_rate = 1e-3,
    metrics=["accuracy"]
)
tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)

2024-01-22 14:07:31,021 - {pytorch_tabular.tabular_model:140} - INFO - Experiment Tracking is turned off

# Step3 & Step 4: Training and Evaluating 

In [15]:
tabular_model.fit(train=train, validation=test)

Seed set to 42


2024-01-22 14:08:45,523 - {pytorch_tabular.tabular_model:524} - INFO - Preparing the DataLoaders

2024-01-22 14:08:45,527 - {pytorch_tabular.tabular_datamodule:499} - INFO - Setting up the datamodule for          
classification task

2024-01-22 14:08:45,590 - {pytorch_tabular.tabular_model:574} - INFO - Preparing the Model: CategoryEmbeddingModel

2024-01-22 14:08:45,910 - {pytorch_tabular.tabular_model:340} - INFO - Preparing the Trainer

...

┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name             ┃ Type                      ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ _backbone        │ CategoryEmbeddingBackbone │ 19.0 M │
│ 1 │ _embedding_layer │ Embedding1dLayer          │     60 │
│ 2 │ head             │ LinearHead                │  1.0 K │
│ 3 │ loss             │ CrossEntropyLoss          │      0 │
└───┴──────────────────┴───────────────────────────┴────────┘

Trainable params: 19.0 M                                                                                           
Non-trainable params: 0                                                                                            
Total params: 19.0 M                                                                                               
Total estimated model params size (MB): 76

Output()

2024-01-22 14:10:28,475 - {pytorch_tabular.tabular_model:663} - INFO - Training the model completed

2024-01-22 14:10:28,477 - {pytorch_tabular.tabular_model:1489} - INFO - Loading the best model

In [16]:
tabular_model.evaluate(test)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │     0.982300877571106     │
│         test_loss         │    0.13017460703849792    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.13017460703849792, 'test_accuracy': 0.982300877571106}]

# Step 5: Saving the Model

In [10]:
tabular_model.save_model('./saved_models/tab_model')

In [11]:
loaded_model = TabularModel.load_from_checkpoint('./saved_models/tab_model/')

# Testing on other datasets

# Adult income dataset

In [19]:
data2=pd.read_csv('input/adult-census-income/adult.csv')

In [24]:
data2.head(3)

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K


In [25]:
data2.drop(['fnlwgt'],axis=1,inplace=True)
data2['income'] = data2['income'].astype('category').cat.codes# encoding labels

In [26]:
train, test, target_col, cat_col_names, num_col_names  = load_classification_data(data2,'income',0.2)

In [29]:
data_config = DataConfig(
    target=['target'], #target should always be a list. Multi-targets are only supported for regression. Multi-Task Classification is not implemented
    continuous_cols=num_col_names,
    categorical_cols=cat_col_names,
    continuous_feature_transform="quantile_normal",
    normalize_continuous_features=True
)
trainer_config = TrainerConfig(
    auto_lr_find=True, # Runs the LRFinder to automatically derive a learning rate
    batch_size=16,
    max_epochs=100,
    auto_select_gpus = True,
    devices=1, #index of the GPU to use. 0, means CPU
)
optimizer_config = OptimizerConfig()
model_config = CategoryEmbeddingModelConfig(
    task="classification",
    layers="4096-4096-512",  # Number of nodes in each layer
    activation="LeakyReLU", # Activation between each layers
    learning_rate = 1e-3,
    metrics=["accuracy"]
)
tabular_model2 = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)

2024-01-22 14:25:54,504 - {pytorch_tabular.tabular_model:140} - INFO - Experiment Tracking is turned off

In [30]:
tabular_model2.fit(train=train, validation=test)

Seed set to 42


2024-01-22 14:26:14,292 - {pytorch_tabular.tabular_model:524} - INFO - Preparing the DataLoaders

2024-01-22 14:26:14,327 - {pytorch_tabular.tabular_datamodule:499} - INFO - Setting up the datamodule for          
classification task

ValueError: could not convert string to float: '11th'

# 🌹 iris-flower dataset

In [12]:
# reading data
data2=pd.read_csv('../input/iris/Iris.csv')
data2.drop(['Id'],axis=1,inplace=True)
data2['Species'] = data2['Species'].astype('category').cat.codes# encoding labels

In [13]:
train, test, target_col, cat_col_names, num_col_names  = load_classification_data(data2,'Species',0.2)

In [14]:
data_config = DataConfig(
    target=['target'], #target should always be a list. Multi-targets are only supported for regression. Multi-Task Classification is not implemented
    continuous_cols=num_col_names,
    categorical_cols=cat_col_names,
    continuous_feature_transform="quantile_normal",
    normalize_continuous_features=True
)
trainer_config = TrainerConfig(
    auto_lr_find=True, # Runs the LRFinder to automatically derive a learning rate
    batch_size=16,
    max_epochs=100,
    auto_select_gpus = True,
    gpus=1, #index of the GPU to use. 0, means CPU
)
optimizer_config = OptimizerConfig()
model_config = CategoryEmbeddingModelConfig(
    task="classification",
    layers="4096-4096-512",  # Number of nodes in each layer
    activation="LeakyReLU", # Activation between each layers
    learning_rate = 1e-3,
    metrics=["accuracy"]
)
tabular_model2 = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)

In [15]:
tabular_model2.fit(train=train, test=test)

Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
-------------------------

Validation sanity check: 0it [00:00, ?it/s]

Training: 1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [16]:
tabular_model2.evaluate(test)

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_accuracy': 1.0}
--------------------------------------------------------------------------------


[{'test_accuracy': 1.0}]

# 👨‍🔬 pima-indian-diabetes dataset

In [17]:
# reading data
data3=pd.read_csv('../input/pima-indians-diabetes-database/diabetes.csv')
# data3.drop('id',axis=1,inplace=True)
data3['quality'] = data3['Outcome'].astype('category').cat.codes# encoding labels

In [18]:
train, test, target_col, cat_col_names, num_col_names= load_classification_data(data3,'quality',0.2)

In [19]:
data_config = DataConfig(
    target=['target'], #target should always be a list. Multi-targets are only supported for regression. Multi-Task Classification is not implemented
    continuous_cols=num_col_names,
    categorical_cols=cat_col_names,
    continuous_feature_transform="quantile_normal",
    normalize_continuous_features=True
)
trainer_config = TrainerConfig(
    auto_lr_find=True, # Runs the LRFinder to automatically derive a learning rate
    batch_size=32,
    max_epochs=100,
    gpus=1, #index of the GPU to use. 0, means CPU
)
optimizer_config = OptimizerConfig()
model_config = CategoryEmbeddingModelConfig(
    task="classification",
    layers="1024-1024-32",  # Number of nodes in each layer
    activation="LeakyReLU", # Activation between each layers
    learning_rate = 1e-3,
    metrics=["accuracy"]
)
tabular_model3 = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)

In [20]:
tabular_model3.fit(train=train, test=test)

Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
-------------------------

Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [21]:
tabular_model3.evaluate(test)

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_accuracy': 0.6993464231491089}
--------------------------------------------------------------------------------


[{'test_accuracy': 0.6993464231491089}]

# ❤ heart-disease-uci dataset

In [22]:
# reading data
data4=pd.read_csv('../input/heart-disease-uci/heart.csv')
# data3.drop('id',axis=1,inplace=True)
data4['target'] = data4['target'].astype('category').cat.codes# encoding labels

In [23]:
train, test, target_col, cat_col_names, num_col_names= load_classification_data(data4,'target',0.2)

In [24]:
data_config = DataConfig(
    target=['target'], #target should always be a list. Multi-targets are only supported for regression. Multi-Task Classification is not implemented
    continuous_cols=num_col_names,
    categorical_cols=cat_col_names,
    continuous_feature_transform="quantile_normal",
    normalize_continuous_features=True
)
trainer_config = TrainerConfig(
    auto_lr_find=True, # Runs the LRFinder to automatically derive a learning rate
    batch_size=32,
    max_epochs=100,
    gpus=1, #index of the GPU to use. 0, means CPU
)
optimizer_config = OptimizerConfig()
model_config = CategoryEmbeddingModelConfig(
    task="classification",
    layers="4096-4096-512",  # Number of nodes in each layer
    activation="LeakyReLU", # Activation between each layers
    learning_rate = 1e-3,
    metrics=["accuracy"]
)
tabular_model4 = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)

In [25]:
tabular_model4.fit(train=train, test=test)

Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
-------------------------

Validation sanity check: 0it [00:00, ?it/s]

Training: 3it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [26]:
tabular_model4.evaluate(test)

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_accuracy': 0.7333333492279053}
--------------------------------------------------------------------------------


[{'test_accuracy': 0.7333333492279053}]

# 🍷 winequality-red dataset

In [27]:
# reading data
data5=pd.read_csv('../input/red-wine-quality-cortez-et-al-2009/winequality-red.csv')
data5['quality'] = data5['quality'].astype('category').cat.codes# encoding labels

In [28]:
train, test, target_col, cat_col_names, num_col_names= load_classification_data(data5,'quality',0.2)

In [29]:
data_config = DataConfig(
    target=['target'], #target should always be a list. Multi-targets are only supported for regression. Multi-Task Classification is not implemented
    continuous_cols=num_col_names,
    categorical_cols=cat_col_names,
    continuous_feature_transform="quantile_normal",
    normalize_continuous_features=True
)
trainer_config = TrainerConfig(
    auto_lr_find=True, # Runs the LRFinder to automatically derive a learning rate
    batch_size=32,
    max_epochs=100,
    gpus=1, #index of the GPU to use. 0, means CPU
)
optimizer_config = OptimizerConfig()
model_config = CategoryEmbeddingModelConfig(
    task="classification",
    layers="4096-4096-512",  # Number of nodes in each layer
    activation="LeakyReLU", # Activation between each layers
    learning_rate = 1e-3,
    metrics=["accuracy"]
)
tabular_model5 = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)

In [30]:
tabular_model5.fit(train=train, test=test)

Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{}
--------------------------------------------------------------------------------


Validation sanity check: 0it [00:00, ?it/s]

Training: 30it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [31]:
tabular_model5.evaluate(test)

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_accuracy': 0.5329153537750244}
--------------------------------------------------------------------------------


[{'test_accuracy': 0.5329153537750244}]

# Conclusion:
The module worked, almost ok. <br>
There is still a lot of research going on in this field, and I don't want to make a prejudice that it won't get any better.<br>
PS: I tried it on [tabular-playground-series-jun-2021](https://www.kaggle.com/c/tabular-playground-series-jun-2021/overview) dataset <br>and it worked so really well and yes it worked better than Classical Algorithms without any preprocessing.

<strong><span style="color:crimson"><span style="font-size:150%">If you like my work, please don't forget leave an upvote!</span></span></strong>

<strong><span style="color: seagreen"><span style="font-size:150%"> If you don't, atleast leave a comment on what should I do to improve it!</span></span></strong>